In [2]:
import os

import pandas as pd
from pandas import DataFrame
import numpy as np
from scipy.stats.mstats import winsorize

from Constants import Constants as const

In [2]:
pc3_df: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'firm_year_num_political_donar_3years.dta')).rename(
    columns={'recipient_ext_id': 'numDonation3year'}).drop(['index'], axis=1)
pc4_df: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'firm_year_num_political_donar_4year.dta')).rename(
    columns={'recipient_ext_id': 'numDonation4year'}).drop(['index'], axis=1)

In [7]:
pc3_avg = pc3_df.groupby([const.GVKEY])['numDonation3year'].mean()
pc4_avg = pc4_df.groupby([const.GVKEY])['numDonation4year'].mean()

In [9]:
pc3_avg_df: DataFrame = pc3_avg.reset_index(drop=False)
pc4_avg_df: DataFrame = pc4_avg.reset_index(drop=False)

In [10]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v2.dta'))
gvkey_series = reg_df[const.GVKEY].unique()

C:\Users\wangy\AppData\Local\Temp\ipykernel_20864\88727916.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241014_stock_act_reg_data_v2.dta'))


In [13]:
# Update the function to handle numpy.ndarray as input
def ensure_gvkeys_with_ndarray(df, gvkey_array, num_col_name):
    # Convert the numpy array to a set for quick lookup
    gvkey_set = set(gvkey_array)
    # Identify missing gvkeys
    existing_gvkeys = set(df['gvkey'])
    missing_gvkeys = gvkey_set - existing_gvkeys
    # Create a DataFrame with missing gvkeys
    missing_rows = pd.DataFrame({
        'gvkey': list(missing_gvkeys),
        num_col_name: [None] * len(missing_gvkeys)
    })
    # Concatenate the original DataFrame with missing rows and sort by gvkey
    updated_df = pd.concat([df, missing_rows], ignore_index=True)
    updated_df = updated_df.sort_values('gvkey').reset_index(drop=True)
    return updated_df

In [14]:
# Update pc3_avg_df and pc4_avg_df
pc3_avg_df2 = ensure_gvkeys_with_ndarray(pc3_avg_df, gvkey_series, 'numDonation3year')
pc4_avg_df2 = ensure_gvkeys_with_ndarray(pc4_avg_df, gvkey_series, 'numDonation4year')


C:\Users\wangy\AppData\Local\Temp\ipykernel_20864\1236285319.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([df, missing_rows], ignore_index=True)


In [16]:
pc_avg_df: DataFrame = pc3_avg_df2.merge(pc4_avg_df2, on=[const.GVKEY])
pc_avg_df.shape

(1837, 3)

In [17]:
for key in ['numDonation3year', 'numDonation4year']:
    pc_avg_df[key] = pc_avg_df[key].fillna(0)

In [20]:
pc_avg_df['highDonation3Year'] = (pc_avg_df['numDonation3year'] > pc_avg_df['numDonation3year'].median()).astype(int)
pc_avg_df['highDonation4Year'] = (pc_avg_df['numDonation4year'] > pc_avg_df['numDonation4year'].median()).astype(int)

In [22]:
reg_df2: DataFrame = reg_df.merge(pc_avg_df, on=[const.GVKEY], how='left')
reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20241212_stock_act_reg_data.dta'), write_index=False, version=119)

# Append Government Contract data

In [2]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241212_stock_act_reg_data.dta'))
gov_contract_df: DataFrame = pd.read_csv(os.path.join(const.DATA_PATH, 'fromZGY', 'gov_characters_data.csv')).drop(['tic'], axis=1)

In [3]:
gov_contract_df.rename(columns=lambda x: '{}_num'.format(x) if x.endswith('gov') else x, inplace=True)

In [4]:
gov_contract_df

,fiscal_year,fed_gov_num,gov_num,state_gov_num,loc_gov_num,gvkey
0,2008.0,0.0,0.0,0.0,0.0,151832
1,2009.0,0.0,0.0,0.0,0.0,151832
2,2010.0,0.0,0.0,0.0,0.0,151832
3,2011.0,0.0,0.0,0.0,0.0,151832
4,2012.0,0.0,0.0,0.0,0.0,151832
...,...,...,...,...,...,...
85811,2011.0,0.0,0.0,0.0,0.0,30165
85812,2012.0,0.0,0.0,0.0,0.0,30165
85813,2013.0,0.0,0.0,0.0,0.0,30165
85814,2014.0,0.0,0.0,0.0,0.0,30165


In [10]:
reg_df_gov_num = reg_df.merge(gov_contract_df, on=[const.GVKEY, const.YEAR], how='left')
for key in gov_contract_df.keys():
    if key not in {const.GVKEY, const.YEAR}:
        reg_df_gov_num[key].fillna(0, inplace=True)
        reg_df_gov_num[key.replace('num', 'dummy')] = reg_df_gov_num[key].apply(lambda x: int(x > 0))

C:\Users\wangy\AppData\Local\Temp\ipykernel_28288\2904554935.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reg_df_gov_num[key].fillna(0, inplace=True)
C:\Users\wangy\AppData\Local\Temp\ipykernel_28288\2904554935.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [11]:
reg_df_gov_num.describe()

,index,gvkey,fiscal_year,mf_indicator,frequency,log_frequency,MajorGovCustomer,post,log_market_value,lev,...,highDonation3Year,highDonation4Year,fed_gov_num,gov_num,state_gov_num,loc_gov_num,fed_gov_dummy,gov_dummy,state_gov_dummy,loc_gov_dummy
count,12074.00000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,...,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000,12074.000000
mean,6036.50000,81220.205317,2011.546960,0.363508,4.817459,0.868412,0.153553,0.508945,6.517410,0.181687,...,0.439043,0.465629,0.263541,0.295511,0.023025,0.008945,0.152725,0.160593,0.016316,0.006957
min,0.00000,1004.000000,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.786519,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3018.25000,12142.000000,2010.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.012064,0.000930,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6036.50000,62592.000000,2012.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.520895,0.148584,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9054.75000,156153.000000,2014.000000,1.000000,7.000000,2.079442,0.000000,1.000000,7.976261,0.264176,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,12073.00000,296753.000000,2015.000000,1.000000,36.000000,3.610918,1.000000,1.000000,11.561005,17.825450,...,1.000000,1.000000,19.000000,19.000000,10.000000,3.000000,1.000000,1.000000,1.000000,1.000000
std,3485.60791,73644.923742,2.279738,0.481029,8.551352,1.245300,0.360535,0.499941,2.163776,0.273871,...,0.496291,0.498838,0.862415,0.934783,0.219107,0.114772,0.359737,0.367170,0.126693,0.083122


In [12]:
reg_df_gov_num.to_stata(os.path.join(const.RESULT_PATH, '20241223_stock_act_reg_data.dta'), write_index=False, version=119)

# append some annual data

In [50]:
reg_df: DataFrame = pd.read_stata(os.path.join(const.RESULT_PATH, '20241223_stock_act_reg_data.dta'))
gov_contract: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'gov_contract.dta')).rename(
    columns={'year': const.YEAR, 'if_gov_contract': 'hasGovContractPanel'}).drop(
    ['index', 'if_cpdata', 'if_costplus', 'if_cas', 'if_noncomm'], axis=1).dropna(how='any').drop_duplicates(
    subset=[const.GVKEY, const.YEAR], keep='first')
govsales_percentage: DataFrame = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'govsales_percentage.dta')).rename(
    columns={'tic': const.TICKER, 'annual_per_sale': 'GovSalePercent'}).drop(
    ['index', 'gov_annual_sales', 'all_annual_sales'], axis=1).dropna(how='any')



In [51]:
dollar_amount = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'fec_2008_2015_firmlevel_dollarnum.dta')).rename(
    columns={'year': const.YEAR, 'amount': 'contrAmt'}).drop(['index'], axis=1).dropna(how='any')
people_num = pd.read_stata(os.path.join(const.DATA_PATH, 'fromZGY', 'fec_2008_2015_firmlevel_peoplenum.dta')).rename(
    columns={'year': const.YEAR, 'recipient_ext_id': 'numPolitician'}).drop(['index'], axis=1).dropna(how='any')

In [52]:
gov_contract[const.YEAR] = gov_contract[const.YEAR].astype(int)
govsales_percentage[const.YEAR] = govsales_percentage[const.YEAR].astype(int)
dollar_amount[const.YEAR] = dollar_amount[const.YEAR].astype(int)
people_num[const.YEAR] = people_num[const.YEAR].astype(int)
gov_contract[const.GVKEY] = gov_contract[const.GVKEY].astype(int)
dollar_amount[const.GVKEY] = dollar_amount[const.GVKEY].astype(int)
people_num[const.GVKEY] = people_num[const.GVKEY].astype(int)

In [63]:
reg_df2: DataFrame = reg_df.merge(govsales_percentage, on=[const.TICKER, const.YEAR], how='left').merge(
    gov_contract, on=[const.GVKEY, const.YEAR], how='left').merge(
    dollar_amount, on=[const.GVKEY, const.YEAR], how='left').merge(
    people_num, on=[const.GVKEY, const.YEAR], how='left')
reg_df2.loc[:, 'hasGovContractPanel'] = reg_df2['hasGovContractPanel'].fillna(0)


In [47]:
reg_df2[['hasGovContract', 'hasGovContractPanel']].describe()

,hasGovContract,hasGovContractPanel
count,12074.000000,12074.000000
mean,0.281762,0.430098
std,0.449877,0.495110
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [64]:
reg_df2['hasGovPanelMajor'] = reg_df2.apply(lambda x: int((x['hasGovContractPanel'] == 1) and (x['MajorGovCustomer'] == 0)), axis=1)
reg_df2['hasGovPanelFed'] = reg_df2.apply(lambda x: int((x['hasGovContractPanel'] == 1) and (x['fed_gov_dummy'] == 0)), axis=1)

In [55]:
reg_df2.loc[:, ['GovSalePercent', 'contrAmt', 'numPolitician']].describe()

,GovSalePercent,contrAmt,numPolitician
count,1362.000000,4.984000e+03,4984.000000
mean,0.421257,3.017618e+04,33.989767
std,0.341576,1.046634e+05,114.451020
min,0.001727,-6.618000e+03,1.000000
25%,0.104907,1.250000e+03,2.000000
50%,0.327493,5.300000e+03,6.000000
75%,0.694477,2.264300e+04,22.000000
max,1.000000,3.602357e+06,2581.000000


In [56]:
def create_high_low_dummy(df: DataFrame, check_key: str):
    df_median = df[check_key].median()
    df[f'High{check_key}'] = 0
    df[f'Low{check_key}'] = 0
    df.loc[df[check_key] > df_median, f'High{check_key}'] = 1
    df.loc[df[check_key] < df_median, f'Low{check_key}'] = 1

    return df


In [65]:
reg_df3 = reg_df2.copy()
for key in ['GovSalePercent', 'contrAmt', 'numPolitician']:
    reg_df3: DataFrame = reg_df3.groupby(const.YEAR).apply(create_high_low_dummy, check_key=key).reset_index(drop=True)


C:\Users\wangy\AppData\Local\Temp\ipykernel_33884\690486281.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reg_df3: DataFrame = reg_df3.groupby(const.YEAR).apply(create_high_low_dummy, check_key=key).reset_index(drop=True)
C:\Users\wangy\AppData\Local\Temp\ipykernel_33884\690486281.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reg_df3: DataFrame = reg_df3.groupby(const.YEAR).apply(create_high_low

In [71]:
reg_df3['GovSalePercent'] = reg_df3['GovSalePercent'].fillna(0)
reg_df3['contrAmt'] = reg_df3['contrAmt'].fillna(0)
reg_df3['numPolitician'] = reg_df3['numPolitician'].fillna(0)

In [72]:
reg_df3.to_stata(os.path.join(const.RESULT_PATH, '20241231_stock_act_data.dta'), write_index=False, version=117)

# Prepare regression data for totalP and disclosure

In [45]:
main_reg_df = pd.read_stata(os.path.join(const.RESULT_PATH, '20250704_stock_act_reg_data_v1.dta'))
placebo_reg_df = pd.read_stata(os.path.join(const.RESULT_PATH, '20250516_2002_2010_regression_data.dta'))

const.YEAR

'fiscal_year'

In [46]:
placebo_reg_df = placebo_reg_df[placebo_reg_df['gov_counts'].notnull()].copy()
usekey_list = 'log_frequency gov_indicator sic fiscal_year gvkey Size LEV BM ROA GuidanceForecast'.split(' ')
placebo_reg_df_useful = placebo_reg_df[usekey_list].rename(columns={'gov_indicator': 'MajorGovCustomer', 'log_frequency': 'logGuidanceForecast'})

In [47]:
main_reg_useful_df = main_reg_df[['frequency', 'log_frequency_w', 'MajorGovCustomer', 'log_market_value_w', 'lev_w', 'BM_w', 'ROA_w', 'sic', 'fiscal_year', 'gvkey']].rename(
    columns={'log_frequency_w': 'logGuidanceForecast', 'frequency': 'GuidanceForecast', 'log_market_value_w': "Size", 'lev_w': 'LEV', 'BM_w': 'BM', 'ROA_w': 'ROA'})
merged_df = pd.concat([placebo_reg_df_useful, main_reg_useful_df], axis=0, ignore_index=True).drop_duplicates(subset=[const.GVKEY, const.YEAR], keep='last')

In [48]:
pc_df = pd.read_stata(os.path.join(const.DATA_PATH, '20190217_FEC_general_firm_year_interaction_newly_without_duplicates_interactions3.dta'))
pc_valid_df = pc_df[['CSTAT_gvkey', 'year', 'FEC_LoseP', 'FEC_WonP', 'FEC_TotalP']].rename(
    columns={'CSTAT_gvkey': const.GVKEY, 'year': const.YEAR, 'FEC_LoseP': 'LoseP', 'FEC_WonP': 'WinP', 'FEC_TotalP': 'TotalP'})
pc_valid_df.loc[:, 'NumP'] = pc_valid_df['WinP'] + pc_valid_df['LoseP']
pc_valid_df['WinRatio'] = pc_valid_df['WinP'] / pc_valid_df['NumP']
pc_valid_df['LoseRatio'] = pc_valid_df['LoseP'] / pc_valid_df['NumP']

In [51]:
tmp_pc_df = pc_valid_df.copy()
pc_dis_df = merged_df.merge(tmp_pc_df, on=[const.GVKEY, const.YEAR], how='right')
pc_dis_df['sic'] = pc_dis_df['sic'].astype(str)
pc_dis_df.to_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_pc_reg_data.dta'), write_index=False)

In [15]:
tmp_pc_df = pc_valid_df.copy()
tmp_pc_df.loc[:, const.YEAR] -= 1
pc_dis_df = merged_df.merge(tmp_pc_df, on=[const.GVKEY, const.YEAR], how='inner')
pc_dis_df['sic'] = pc_dis_df['sic'].astype(int)
pc_dis_df.to_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_pc_reg_data_v2.dta'), write_index=False)

In [20]:
def get_election_cycle(fiscal_year):
    if fiscal_year % 2 == 0:
        return fiscal_year - 2
    else:
        return fiscal_year - 1

tmp_pc_df = pc_valid_df.rename(columns={const.YEAR: 'ElectionCycle'})
merged_df.loc[:, 'ElectionCycle'] = merged_df[const.YEAR].apply(get_election_cycle)

pc_dis_df = merged_df.merge(tmp_pc_df, on=[const.GVKEY, 'ElectionCycle'], how='inner')
pc_dis_df['sic'] = pc_dis_df['sic'].astype(int)
pc_dis_df.to_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_pc_reg_data_v3.dta'), write_index=False)


In [21]:
tmp_pc_df = pc_valid_df.rename(columns={const.YEAR: 'ElectionCycle'})
tmp_dis_df = merged_df.groupby([const.GVKEY, 'ElectionCycle'])[['Size', 'LEV', 'BM', 'ROA', 'GuidanceForecast']].mean().reset_index(drop=False)
pc_dis_df = tmp_dis_df.merge(tmp_pc_df, on=[const.GVKEY, 'ElectionCycle'], how='inner')
pc_dis_df['logGuidanceForecast'] = pc_dis_df['GuidanceForecast'].apply(np.log1p)
pc_dis_df.to_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_pc_reg_data_v4.dta'), write_index=False)

## merge with special election data

In [43]:
spc_df = pd.read_stata(os.path.join(const.DATA_PATH, '20180829_FEC_federal_special_firm_year_candidate_merged_data.dta'))
useful_col = 'year FEC_margin FEC_is_win FEC_is_close_election CSTAT_gvkey'.split(' ')
spc_df = spc_df[useful_col].rename(columns={'year': const.YEAR, 'CSTAT_gvkey': const.GVKEY})

In [53]:
spc_dis_df = merged_df.merge(spc_df, on=[const.GVKEY, const.YEAR], how='right')
spc_dis_df.describe()
spc_dis_df['sic'] = spc_dis_df['sic'].astype(str)
spc_dis_df.drop_duplicates(subset=[const.GVKEY, const.YEAR], keep='last').to_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_spc_reg_data.dta'), write_index=False)

# Construct Regression data for the following variables
## GPIN

In [15]:
crsp_comp_link = pd.read_pickle(os.path.join(const.TEMP_PATH, 'crsp_comp.pkl'))
crsp_comp_link['permco'] = crsp_comp_link['permco'].astype(int)
crsp_comp_link[const.GVKEY] = crsp_comp_link[const.GVKEY].astype(int)

In [4]:
cpin_daily_full = pd.read_csv(os.path.join(const.RESULT_PATH, '1993_2019_cpie_daily.csv'))
annual_gpin = cpin_daily_full.groupby(['permno', 'year']).agg({
    'cpie_gpin': 'mean',
}).reset_index(drop=False)
annual_gpin.rename(columns={'year': const.YEAR}, inplace=True)

In [16]:
annual_gpin_gvkey = pd.merge(annual_gpin, crsp_comp_link,
                             left_on='permno', right_on='permco', how='left')

## IDIOSYN

In [5]:
synchrony_monthly_df = pd.read_pickle(os.path.join(const.TEMP_PATH, '20250712_synchrony_monthly.pkl'))
synchrony_monthly_df['IDIOSYN'] = synchrony_monthly_df['SYNCHRONICITY'] * -1

In [18]:
synchrony_gvkey_df = pd.merge(synchrony_monthly_df.drop(['SYNCHRONICITY', 'SYNCHRONICITY_MKT', 'SYNCHRONICITY_IND'], axis=1),
                                crsp_comp_link,
                             left_on='PERMNO', right_on='permco', how='left')

In [5]:
idiosyn_df = pd.read_pickle(os.path.join(const.TEMP_PATH, '20250717_idiosyn_monthly.pkl'))
idiosyn_df.describe()

,PERMNO,year,IDIOSYN
count,444782.000000,444782.000000,385109.000000
mean,52339.816402,1994.478781,-1.445707
std,29679.843760,20.945936,1.985146
min,10000.000000,1926.000000,-36.736801
25%,20722.000000,1982.000000,-2.151028
50%,54317.000000,1997.000000,-1.271565
75%,80830.000000,2011.000000,-0.478086
max,93436.000000,2024.000000,8.074132


## DELAY

In [6]:
price_delay_df = pd.read_pickle(os.path.join(const.TEMP_PATH, 'hm2005_all_price_delay.pkl'))

In [5]:
price_delay_df.keys()

Index(['PERMNO', 'year', 'price_delay', 'r2_full', 'r2_restricted', 'n_obs'], dtype='object')

In [6]:
price_delay_df.describe()

,PERMNO,year,price_delay,r2_full,r2_restricted,n_obs
count,435361.000000,435361.000000,433884.000000,433884.000000,4.338840e+05,435361.000000
mean,52247.041577,1994.373591,0.474898,0.269389,1.803564e-01,45.998470
std,29631.207052,21.023619,0.327104,0.201966,2.088700e-01,7.169638
min,10000.000000,1926.000000,0.000000,0.000740,1.197764e-11,10.000000
25%,20748.000000,1982.000000,0.171768,0.119707,2.459033e-02,48.000000
50%,54114.000000,1997.000000,0.430954,0.213365,1.018642e-01,48.000000
75%,80723.000000,2011.000000,0.784484,0.363214,2.621346e-01,48.000000
max,93436.000000,2024.000000,1.000000,1.000000,1.000000e+00,49.000000


In [28]:
delay_gvkey_df = pd.merge(price_delay_df, crsp_comp_link,
                          left_on='PERMNO', right_on='permco', how='left').dropna(subset=[const.GVKEY]).rename(columns={'year': const.YEAR}).drop(['PERMNO', 'r2_full',  'r2_restricted', 'n_obs', 'permco'], axis=1)

## ANALYSTS, FCSTERROR and DISPERSION

In [3]:
afd_df = pd.read_pickle(os.path.join(const.TEMP_PATH, '1976_2023_analysts_fd_data.pkl'))
afd_df.describe()

,fiscal_year,eps_mean,eps_sd,eps_actual,numest,eps_mean_last,eps_sd_last,eps_actual_last,numest_last,permno,...,mthprevprc,mthret,vwretd,date,Price_lag,DISPERSION,DISPERSION_last,FCSTERROR,FCSTERROR_last,ANALYSTS
count,172208.000000,1.721590e+05,1.473560e+05,1.670650e+05,172208.000000,1.721590e+05,1.473560e+05,1.670650e+05,172208.000000,172208.000000,...,172131.000000,172127.000000,172208.000000,172208,172208.000000,1.473560e+05,1.473560e+05,1.670360e+05,1.670360e+05,0.0
mean,2001.524035,2.092903e+04,1.918929e+05,-1.553316e+06,6.669149,-1.270912e+04,1.494321e+05,-1.534378e+06,6.642345,59601.252340,...,50.250090,0.015151,0.017939,2002-06-30 22:38:55.835733504,47.237687,8.673160e+04,5.574889e+04,6.038095e+04,5.865943e+04,NaN
min,1977.000000,-9.750000e+08,0.000000e+00,-3.564587e+10,1.000000,-9.882353e+08,0.000000e+00,-3.763200e+10,1.000000,10001.000000,...,0.015625,-1.000000,-0.225361,1977-12-30 00:00:00,0.020000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
25%,1993.000000,1.958333e-01,2.916667e-02,6.833333e-02,1.916667,1.600000e-01,2.000000e-02,6.020000e-02,2.000000,30728.000000,...,7.812500,-0.048321,0.003676,1993-12-31 00:00:00,8.375000,1.230753e-03,7.390300e-04,2.003949e-03,1.181536e-03,NaN
50%,2001.000000,7.983333e-01,6.583333e-02,6.958333e-01,4.166667,7.800000e-01,5.000000e-02,7.000000e-01,4.000000,75055.000000,...,17.670000,0.010870,0.018294,2001-12-31 00:00:00,17.660000,3.423006e-03,2.305919e-03,7.588250e-03,5.151602e-03,NaN
75%,2011.000000,1.740000e+00,1.641964e-01,1.645000e+00,9.000000,1.740000e+00,1.300000e-01,1.670000e+00,9.000000,84001.000000,...,32.500000,0.072959,0.031839,2011-12-30 00:00:00,31.642500,1.074177e-02,8.212652e-03,3.143124e-02,2.495423e-02,NaN
max,2021.000000,4.800000e+09,4.806799e+09,4.080000e+09,54.666667,4.800000e+09,5.623709e+09,4.080000e+09,57.000000,93436.000000,...,416876.000000,15.774194,0.129674,2021-12-31 00:00:00,347815.000000,2.247453e+09,1.739205e+09,2.482667e+09,2.900800e+09,NaN
std,11.561869,1.646140e+07,2.312975e+07,1.731301e+08,6.704304,1.438131e+07,2.160761e+07,1.745210e+08,6.810433,28512.242719,...,2305.437187,0.173908,0.035051,NaN,2083.659628,1.038527e+07,7.201449e+06,8.490775e+06,9.372193e+06,NaN


## Guaidance RangeForecast

In [14]:
guidance_df = pd.read_pickle(os.path.join(const.TEMP_PATH, '20250718_ibes_firm_year_guidance.pkl'))
guidance_df.describe()

,fiscal_year,guidancecount,rangeforecast
count,60140.000000,60140.000000,48383.000000
mean,2009.203475,9.603991,0.796553
std,6.836092,9.613754,0.244476
min,1992.000000,1.000000,0.012346
25%,2004.000000,2.000000,0.666667
50%,2009.000000,6.000000,0.892857
75%,2015.000000,14.000000,1.000000
max,2021.000000,131.000000,1.000000


## Merge all five dataset

In [15]:
print(annual_gpin.keys())
print(idiosyn_df.keys())
print(afd_df.keys())
print(price_delay_df.keys())
print(guidance_df.keys())

Index(['permno', 'fiscal_year', 'cpie_gpin'], dtype='object')
Index(['PERMNO', 'year', 'IDIOSYN'], dtype='object')
Index(['permno', 'fiscal_year', 'numest', 'numest_last', 'DISPERSION',
       'DISPERSION_last', 'FCSTERROR', 'FCSTERROR_last'],
      dtype='object')
Index(['PERMNO', 'year', 'price_delay', 'r2_full', 'r2_restricted', 'n_obs'], dtype='object')
Index(['permno', 'fiscal_year', 'guidancecount', 'rangeforecast'], dtype='object')


In [16]:
id_df = idiosyn_df.rename(columns={'year': const.YEAR, 'PERMNO': 'permno'}).dropna(how='any')
afd_df = afd_df[['permno', const.YEAR, 'numest', 'numest_last', 'DISPERSION', 'DISPERSION_last', 'FCSTERROR', 'FCSTERROR_last']].dropna(subset=['permno', const.YEAR], how='any')
pd_df = price_delay_df[['PERMNO', 'year', 'price_delay']].rename(columns={'year': const.YEAR, 'PERMNO': 'permno'}).dropna(how='any')
dep_df = annual_gpin.merge(id_df, on=['permno', const.YEAR], how='outer').merge(
    afd_df, on=['permno', const.YEAR], how='outer').merge(
    pd_df, on=['permno', const.YEAR], how='outer').merge(
    guidance_df, on=['permno', const.YEAR], how='outer')

In [18]:
dep_df.describe()

,fiscal_year,cpie_gpin,IDIOSYN,numest,numest_last,DISPERSION,DISPERSION_last,FCSTERROR,FCSTERROR_last,price_delay,guidancecount,rangeforecast
count,436027.000000,3.659300e+04,385109.000000,172208.000000,172208.000000,1.473560e+05,1.473560e+05,1.670360e+05,1.670360e+05,433884.000000,60140.000000,48383.000000
mean,1994.438402,3.864901e-01,-1.445707,6.669149,6.642345,8.673160e+04,5.574889e+04,6.038095e+04,5.865943e+04,0.474898,9.603991,0.796553
std,21.005916,1.822650e-01,1.985146,6.704304,6.810433,1.038527e+07,7.201449e+06,8.490775e+06,9.372193e+06,0.327104,9.613754,0.244476
min,1926.000000,7.634524e-272,-36.736801,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000,0.012346
25%,1982.000000,2.285479e-01,-2.151028,1.916667,2.000000,1.230753e-03,7.390300e-04,2.003949e-03,1.181536e-03,0.171768,2.000000,0.666667
50%,1997.000000,4.126209e-01,-1.271565,4.166667,4.000000,3.423006e-03,2.305919e-03,7.588250e-03,5.151602e-03,0.430954,6.000000,0.892857
75%,2011.000000,5.244818e-01,-0.478086,9.000000,9.000000,1.074177e-02,8.212652e-03,3.143124e-02,2.495423e-02,0.784484,14.000000,1.000000
max,2024.000000,1.000000e+00,8.074132,54.666667,57.000000,2.247453e+09,1.739205e+09,2.482667e+09,2.900800e+09,1.000000,131.000000,1.000000


In [19]:
dep_df.to_pickle(os.path.join(const.TEMP_PATH, '20250718_temp_dependent_variables.pkl'))

## Construct Regression data

In [3]:
dep_df = pd.read_pickle(os.path.join(const.TEMP_PATH, '20250712_temp_dependent_variables.pkl'))
ccm_path = r'D:\Users\wangy\Documents\data\ccm'
ccm_df = pd.read_sas(os.path.join(os.path.join(ccm_path, 'ccm_lookup.sas7bdat')), encoding='utf-8')

In [4]:
print('dep_df keys: ', list(dep_df.keys()))
print('ccm_df keys: ', list(ccm_df.keys()))

dep_df keys:  ['permno', 'fiscal_year', 'cpie_gpin', 'IDIOSYN', 'coverage', 'price_delay', 'DISPERSION', 'DISPERSION_last', 'FCSTERROR', 'FCSTERROR_last']
ccm_df keys:  ['GVKEY', 'LPERMNO', 'LPERMCO', 'LINKDT', 'LINKENDDT', 'conm', 'tic', 'cusip', 'cik', 'sic', 'naics', 'gsubind', 'gind', 'year1', 'year2']


In [6]:
# 标准化列名
dep_df.columns = dep_df.columns.str.lower()
ccm_df.columns = ccm_df.columns.str.lower()

# 保留需要的字段
ccm_sub = ccm_df[['gvkey', 'lpermno', 'year1', 'year2']].copy()

# 为提高 merge 效率，先 merge on permno，再做年份筛选
merged = pd.merge(dep_df, ccm_sub, left_on='permno', right_on='lpermno', how='left')

# 年份条件筛选：fiscal_year ∈ [year1, year2]
matched = merged[
    (merged['fiscal_year'] >= merged['year1']) &
    (merged['fiscal_year'] <= merged['year2'])
].copy()

# 如果多个匹配，选择最近的 year1（可选）
matched = matched.sort_values(by=['permno', 'fiscal_year', 'year1'], ascending=[True, True, False])
matched = matched.drop_duplicates(subset=['permno', 'fiscal_year'], keep='first')

# 删除辅助列（可选）
matched = matched.drop(columns=['lpermno', 'year1', 'year2'])
matched[const.GVKEY] = matched[const.GVKEY].astype(int)


In [2]:
reg_df = pd.read_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_pc_reg_data_v2.dta'))
spc_reg_df = pd.read_stata(os.path.join(const.RESULT_PATH, '20250711_stock_act_spc_reg_data_v2.dta'))

In [7]:
# merge to next year
tmp_dep_df = matched.copy()
tmp_dep_df['fiscal_year'] = tmp_dep_df['fiscal_year'] + 1

reg_df2 = reg_df.merge(tmp_dep_df, on=[const.GVKEY, const.YEAR], how='left').drop_duplicates(subset=[const.GVKEY, const.YEAR], keep='first')
reg_df2['coverage'] = reg_df2['coverage'].fillna(0)

reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20250713_staock_act_pc_reg_data.dta'), write_index=False)

spc_reg_df2 = spc_reg_df.merge(tmp_dep_df, on=[const.GVKEY, const.YEAR], how='left').drop_duplicates(subset=[const.GVKEY, const.YEAR], keep='first')
spc_reg_df2['coverage'] = spc_reg_df2['coverage'].fillna(0)

spc_reg_df2.to_stata(os.path.join(const.RESULT_PATH, '20250713_staock_act_spc_reg_data.dta'), write_index=False)